In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
import numpy as np
import math

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Load file

In [ ]:
# Cranfiel
import gdown
!gdown 1D48c_crSg6ia_OqZ5HoJmVmDLgn-iqq5
!gdown 1g0JYIiu-l6dSo5BpOuAeem2rVCkxNHjE

Downloading...
From: https://drive.google.com/uc?id=1D48c_crSg6ia_OqZ5HoJmVmDLgn-iqq5
To: /content/Cranfield.zip
100% 851k/851k [00:00<00:00, 29.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1g0JYIiu-l6dSo5BpOuAeem2rVCkxNHjE
To: /content/TEST.zip
100% 49.9k/49.9k [00:00<00:00, 105MB/s]


In [ ]:
# NFCorpus
!gdown 1QscBtuFj3XW7Gl3lhWdEx6r8Yit0yJ2v

Downloading...
From: https://drive.google.com/uc?id=1QscBtuFj3XW7Gl3lhWdEx6r8Yit0yJ2v
To: /content/nfcorpus.tar.gz
100% 97.3M/97.3M [00:01<00:00, 94.4MB/s]


# Reading Files Cranfield



In [ ]:
!unzip 'Cranfield.zip'

Archive:  Cranfield.zip
   creating: Cranfield/
  inflating: Cranfield/1.txt         
  inflating: Cranfield/10.txt        
  inflating: Cranfield/100.txt       
  inflating: Cranfield/1000.txt      
  inflating: Cranfield/1001.txt      
  inflating: Cranfield/1002.txt      
  inflating: Cranfield/1003.txt      
  inflating: Cranfield/1004.txt      
  inflating: Cranfield/1005.txt      
  inflating: Cranfield/1006.txt      
  inflating: Cranfield/1007.txt      
  inflating: Cranfield/1008.txt      
  inflating: Cranfield/1009.txt      
  inflating: Cranfield/101.txt       
  inflating: Cranfield/1010.txt      
  inflating: Cranfield/1011.txt      
  inflating: Cranfield/1012.txt      
  inflating: Cranfield/1013.txt      
  inflating: Cranfield/1014.txt      
  inflating: Cranfield/1015.txt      
  inflating: Cranfield/1016.txt      
  inflating: Cranfield/1017.txt      
  inflating: Cranfield/1018.txt      
  inflating: Cranfield/1019.txt      
  inflating: Cranfield/102.txt       
  

In [ ]:
!unzip 'TEST.zip'

Archive:  TEST.zip
  inflating: query.txt               
   creating: RES/
  inflating: RES/1.txt               
  inflating: RES/10.txt              
  inflating: RES/100.txt             
  inflating: RES/101.txt             
  inflating: RES/102.txt             
 extracting: RES/103.txt             
  inflating: RES/104.txt             
  inflating: RES/105.txt             
  inflating: RES/106.txt             
  inflating: RES/107.txt             
  inflating: RES/108.txt             
  inflating: RES/109.txt             
  inflating: RES/11.txt              
  inflating: RES/110.txt             
  inflating: RES/111.txt             
 extracting: RES/112.txt             
  inflating: RES/113.txt             
  inflating: RES/114.txt             
  inflating: RES/115.txt             
  inflating: RES/116.txt             
 extracting: RES/117.txt             
  inflating: RES/118.txt             
 extracting: RES/119.txt             
  inflating: RES/12.txt              
  inflating: 

In [ ]:
# Loading the queries of Cranfield
def load_queries(file_path):
    queries = {}
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            key, value = line.strip().split('\t')
            queries[int(key)] = value
    return queries

queries = load_queries('/content/query.txt')
queries

{1: 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .',
 2: 'what are the structural and aeroelastic problems associated with flight of high speed aircraft .',
 3: 'what problems of heat conduction in composite slabs have been solved so far .',
 4: 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium .',
 5: 'what chemical kinetic system is applicable to hypersonic aerodynamic problems .',
 6: 'what theoretical and experimental guides do we have as to turbulent couette flow behaviour .',
 7: 'is it possible to relate the available pressure distributions for an ogive forebody at zero angle of attack to the lower surface pressures of an equivalent ogive forebody at angle of attack .',
 8: 'what methods -dash exact or approximate -dash are presently available for predicting body pressures at

In [ ]:
# Reading the Cranfield's documents
import os

directory = '/content/Cranfield'  # Update with the path to your text files
file_extension = '.txt'  # Update with the file extension of your text files

text_files = [file for file in os.listdir(directory) if file.endswith(file_extension)]

data_dict = {}

for file_name in text_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'r') as file:
        contents = file.read()
        key = int(os.path.splitext(file_name)[0])  # Remove the file extension from the key
        data_dict[key] = contents

# Now you have the contents of each text file stored in the 'data_dict' dictionary

sorted_data_dict = dict(sorted(data_dict.items()))

In [ ]:
sorted_data_dict[1]

'experimental investigation of the aerodynamics of a wing in a slipstream . an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem . the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary layer control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory . an empirical evaluation of the destalling effects was made for the specific configuration of the experiment . '

# All the needed functions

## Porter Stemmer Preprocess

In [ ]:
def preprocessPorterStemmerOnly(docs):
    # docs is a dict {id: doc}
    porter_stemmer = PorterStemmer()
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word.isdigit() or word.isnumeric():
                continue
            if not word.isalnum():
                continue
            word = porter_stemmer.stem(word)
            terms.add(word)
            sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms

In [ ]:
# Porter Stemmer version of the Preprocess
def preprocessPorterStemmer(docs):
    # docs is a dict {id: doc}
    porter_stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word not in string.punctuation and word not in stop_words:
                if word.isdigit() or word.isnumeric():
                    continue
                if not word.isalnum():
                    continue
                word = porter_stemmer.stem(word)
                terms.add(word)
                sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms

# docs, terms = preprocessPorterStemmer(docs)

## Lemmatizer Preprocess

In [ ]:
def preprocessLemmatizerOnly(docs):
    # docs is a dict {id: doc}
    lemmatizer = WordNetLemmatizer()
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word.isdigit() or word.isnumeric():
                continue
            if not word.isalnum():
                continue
            word = lemmatizer.lemmatize(word)
            terms.add(word)
            sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms


In [ ]:
# Lemmatizer Version of Preprocess
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string

def preprocessLematizer(docs):
    # docs is a dict {id: doc}
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word not in string.punctuation and word not in stop_words:
                if word.isdigit() or word.isnumeric():
                    continue
                if not word.isalnum():
                    continue
                word = lemmatizer.lemmatize(word)
                terms.add(word)
                sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms

# docs, terms = preprocessLematizer(docs)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Indexing

In [ ]:
# Create the indexing, order term and inverse order term

def indexing(docs, terms):
    # docs is a dict which {id: doc}
    # terms is the set word not include punctuation
    # indexs is a list {'word': doc_id} word can duplicate
    # order_term is a dict {'word': order in term}
    # inverse_order_term {order in term: 'word'}

    index = []
    order_term = {term: index for index, term in enumerate(terms)}
    inverse_order_term = {index: term for term, index in order_term.items()}
    for doc_id, doc in docs.items():
        words = re.findall(r'\b\w+\b', doc)
        words = [word for word in words if word in terms]
        for word in words:
            index.append({'word': word, 'doc_id':doc_id})

    return index, order_term, inverse_order_term

# indexs, order_term, inverse_order_term = indexing(docs, terms)

## Find Max ID

In [ ]:
def find_max_id(docs):
    max_id = 0
    for doc in docs:
        max_id = max(max_id, doc)
    return max_id

# max_id = find_max_id(docs)

## Adding Frequencies

In [ ]:
# Adding Frequency to the index table

def adding_frequency(docs, terms, sorted_indexs):
    # new_sorted_indexs is a dict {'word', doc_id, frequency in doc_id}
    new_sorted_indexs = []
    for entry in sorted_indexs:
        word = entry['word']
        doc_id = entry['doc_id']
        frequency = docs[doc_id].count(word)
        entry['frequency'] = frequency
        new_sorted_indexs.append(entry)
    return new_sorted_indexs

# new_sorted_indexs = adding_frequency(docs, terms, sorted_indexs)

## Build Vocabs

In [ ]:
# Build the vocabs

def build_vocab(docs, terms, new_sorted_indexs):
    # vocab {'word': word, 'number of doc include': number of document, 'frequency': frequency, 'position' : a set position in new_sorted_indexs}


    vocab = {}
    for i, entry in enumerate(new_sorted_indexs):
        word = entry['word']
        doc_id = entry['doc_id']
        if word not in vocab:
            vocab[word] = {'number of doc include': 0, 'frequency': 0, 'position': set()}
        if doc_id not in docs:
            continue
        vocab[word]['number of doc include'] += 1
        vocab[word]['frequency'] += entry['frequency']
        vocab[word]['position'].add(i)

    return vocab

# vocab = build_vocab(docs, terms, sorted_indexs)

## IDF

In [ ]:
# Calculate the IDF of a term with multiple options

def calculate_idf(docs, terms, vocab, new_sorted_indexs, option):
    if option == 'inverse':
        calculate_inverse_idf(vocab, len(docs))
    elif option == 'count':
        calculate_count_idf(vocab)
    elif option == 'square':
        calculate_inverse_idf(vocab, len(docs))
        calculate_square_idf(vocab)
    elif option == 'proba':
        calculate_proba_idf(vocab, len(docs))
    elif option == 'frequency':
        calculate_frequency_idf(vocab, new_sorted_indexs)
    return vocab

def calculate_inverse_idf(vocab, total_docs):
    for word in vocab:
        num_docs = vocab[word]['number of doc include']
        if num_docs > 0:
            idf = math.log(total_docs / num_docs)
            vocab[word]['idf'] = idf

def calculate_count_idf(vocab):
    for word in vocab:
        num_docs = vocab[word]['number of doc include']
        if num_docs > 0:
            vocab[word]['idf'] = 1 / num_docs

def calculate_square_idf(vocab):
    for word in vocab:
        if 'idf' in vocab[word]:
            idf = vocab[word]['idf']
            square_idf = idf * idf
            vocab[word]['idf'] = square_idf

def calculate_proba_idf(vocab, total_docs):
    for word in vocab:
        num_docs = vocab[word]['number of doc include']
        if num_docs > 0:
            idf = math.log((total_docs - num_docs) / num_docs)
            vocab[word]['idf'] = idf

def calculate_frequency_idf(vocab, new_sorted_indexs):
    for word in vocab:
        num_docs = vocab[word]['number of doc include']
        if num_docs > 0:
            idf = 1 / num_docs
            vocab[word]['idf'] = idf

## TF

In [ ]:
# Calculate the TF of a term with multiple options

def calculate_tf(docs, terms, vocab, new_sorted_indexs, option):
    if option == 'binary':
        calculate_binary_tf(new_sorted_indexs)
    elif option == 'count':
        calculate_count_tf(new_sorted_indexs)
    elif option == 'max':
        calculate_max_tf(new_sorted_indexs)
    elif option == 'augmented':
        calculate_augmented_tf(new_sorted_indexs)
    elif option == 'logarithmic':
        calculate_logarithmic_tf(new_sorted_indexs)
    return new_sorted_indexs

def calculate_binary_tf(new_sorted_indexs):
    for entry in new_sorted_indexs:
        entry['tf'] = 1

def calculate_count_tf(new_sorted_indexs):
    for entry in new_sorted_indexs:
        entry['tf'] = entry['frequency']

def calculate_max_tf(new_sorted_indexs):
    max_tf = max(entry['frequency'] for entry in new_sorted_indexs)
    for entry in new_sorted_indexs:
        entry['tf'] = entry['frequency'] / max_tf

def calculate_augmented_tf(new_sorted_indexs):
    max_tf = max(entry['frequency'] for entry in new_sorted_indexs)
    for entry in new_sorted_indexs:
        entry['tf'] = 0.5 + 0.5 * entry['frequency'] / max_tf

def calculate_logarithmic_tf(new_sorted_indexs):
    for entry in new_sorted_indexs:
        entry['tf'] = 1 + math.log(entry['frequency'])

## Calculate the weight

In [ ]:
def calculate_w(docs, terms, new_idf_vocab, new_tf_sorted_indexs, option):
    # w = idf * tf
    # tf in new_sorted_indexs
    # idf in vocab

    for entry in new_tf_sorted_indexs:
        word = entry['word']
        doc_id = entry['doc_id']
        if word in new_idf_vocab:
            tf = entry['tf']
            idf = new_idf_vocab[word]['idf']
        entry['w'] = idf * tf
    if option == 'sum':
        calculate_sum_w(new_tf_sorted_indexs)
    elif option == 'cosine':
        calculate_cosine_w(new_tf_sorted_indexs)
    elif option == '**4':
        calculate_power_4_w(new_tf_sorted_indexs)
    elif option == 'max':
        calculate_max_w(new_tf_sorted_indexs)

    return new_tf_sorted_indexs

def calculate_sum_w(new_tf_sorted_indexs):
    doc_weights = {}
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        w = entry['w']
        if doc_id in doc_weights:
            doc_weights[doc_id] += w
        else:
            doc_weights[doc_id] = w
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        entry['w'] /= doc_weights[doc_id]

def calculate_cosine_w(new_tf_sorted_indexs):
    doc_lengths = {}
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        w = entry['w']
        if doc_id in doc_lengths:
            doc_lengths[doc_id] += w ** 2
        else:
            doc_lengths[doc_id] = w ** 2
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        w = entry['w']
        length = math.sqrt(doc_lengths[doc_id])
        entry['w'] = w / length if length != 0 else 0.0

def calculate_power_4_w(new_tf_sorted_indexs):
    doc_lengths = {}
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        w = entry['w']
        if doc_id in doc_lengths:
            doc_lengths[doc_id] += w ** 4
        else:
            doc_lengths[doc_id] = w ** 4
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        w = entry['w']
        length = doc_lengths[doc_id]
        entry['w'] = w / length if length != 0 else 0.0

def calculate_max_w(new_tf_sorted_indexs):
    doc_max_w = {}
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        w = entry['w']

        if doc_id in doc_max_w:
            doc_max_w[doc_id] = max(doc_max_w[doc_id], w)
        else:
            doc_max_w[doc_id] = w
    for entry in new_tf_sorted_indexs:
        doc_id = entry['doc_id']
        w = entry['w']
        max_w = doc_max_w[doc_id]
        entry['w'] = w / max_w if max_w != 0 else 0.0

## Calculate the TF-IDF Score of the query

In [ ]:
def calculate_tf_idf_query(query, new_tf_sorted_indexs, new_idf_vocab, order_term, option_norm):
    tf_idf_query = {}
    query = query.lower()
    query = re.findall(r'\b\w+\b', query)
    query = [word for word in query if word in terms]
    # tf number appearance
    # idf get in new_idf_vocab
    # option norm: 'sum', 'cosine', '**4', 'max'

    term_frequencies = {}
    for term in query:
        term_frequencies[term] = term_frequencies.get(term, 0) + 1

    for term, frequency in term_frequencies.items():
        if term in order_term:
            tf = frequency
            idf = new_idf_vocab[term]['idf']
            tf_idf_query[term] = tf * idf

    if option_norm == 'sum':
        tf_idf_sum = sum(tf_idf_query.values())
        if tf_idf_sum != 0:
            tf_idf_query = {term: tf_idf / tf_idf_sum for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'cosine':
        tf_idf_norm = math.sqrt(sum([tf_idf ** 2 for tf_idf in tf_idf_query.values()]))
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == '**4':
        tf_idf_norm = sum([tf_idf ** 4 for tf_idf in tf_idf_query.values()])
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'max':
        tf_idf_max = max(tf_idf_query.values())
        if tf_idf_max != 0:
            tf_idf_query = {term: tf_idf / tf_idf_max for term, tf_idf in tf_idf_query.items()}
    return tf_idf_query

In [ ]:
def calculate_tf_idf_queryPorterStemmerOnly(query, new_tf_sorted_indexs, new_idf_vocab, order_term, option_norm):
    tf_idf_query = {}
    query = query.lower()
    query = re.findall(r'\b\w+\b', query)

    porter_stemmer = PorterStemmer()
    query = [porter_stemmer.stem(word) for word in query]

    # tf number appearance
    # idf get in new_idf_vocab
    # option norm: 'sum', 'cosine', '**4', 'max'

    term_frequencies = {}
    for term in query:
        term_frequencies[term] = term_frequencies.get(term, 0) + 1

    for term, frequency in term_frequencies.items():
        if term in order_term:
            tf = frequency
            idf = new_idf_vocab[term]['idf']
            tf_idf_query[term] = tf * idf

    if option_norm == 'sum':
        tf_idf_sum = sum(tf_idf_query.values())
        if tf_idf_sum != 0:
            tf_idf_query = {term: tf_idf / tf_idf_sum for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'cosine':
        tf_idf_norm = math.sqrt(sum([tf_idf ** 2 for tf_idf in tf_idf_query.values()]))
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == '**4':
        tf_idf_norm = sum([tf_idf ** 4 for tf_idf in tf_idf_query.values()])
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'max':
        tf_idf_max = max(tf_idf_query.values())
        if tf_idf_max != 0:
            tf_idf_query = {term: tf_idf / tf_idf_max for term, tf_idf in tf_idf_query.items()}
    return tf_idf_query

In [ ]:
def calculate_tf_idf_queryPorterStemmer(query, new_tf_sorted_indexs, new_idf_vocab, order_term, option_norm):
    tf_idf_query = {}
    query = query.lower()
    query = re.findall(r'\b\w+\b', query)

    porter_stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    query = [porter_stemmer.stem(word) for word in query if word not in stop_words]

    # tf number appearance
    # idf get in new_idf_vocab
    # option norm: 'sum', 'cosine', '**4', 'max'

    term_frequencies = {}
    for term in query:
        term_frequencies[term] = term_frequencies.get(term, 0) + 1

    for term, frequency in term_frequencies.items():
        if term in order_term:
            tf = frequency
            idf = new_idf_vocab[term]['idf']
            tf_idf_query[term] = tf * idf

    if option_norm == 'sum':
        tf_idf_sum = sum(tf_idf_query.values())
        if tf_idf_sum != 0:
            tf_idf_query = {term: tf_idf / tf_idf_sum for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'cosine':
        tf_idf_norm = math.sqrt(sum([tf_idf ** 2 for tf_idf in tf_idf_query.values()]))
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == '**4':
        tf_idf_norm = sum([tf_idf ** 4 for tf_idf in tf_idf_query.values()])
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'max':
        tf_idf_max = max(tf_idf_query.values())
        if tf_idf_max != 0:
            tf_idf_query = {term: tf_idf / tf_idf_max for term, tf_idf in tf_idf_query.items()}
    return tf_idf_query

# query = "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft ."
# tf_idf_query = calculate_tf_idf_query(query, new_tf_sorted_indexs, new_idf_vocab, order_term, 'sum')

In [ ]:
def calculate_tf_idf_queryLemmatizerOnly(query, new_tf_sorted_indexs, new_idf_vocab, order_term, option_norm):
    tf_idf_query = {}
    query = query.lower()
    query = re.findall(r'\b\w+\b', query)

    lemmatizer = WordNetLemmatizer()
    query = [lemmatizer.lemmatize(word) for word in query]

    # tf number appearance
    # idf get in new_idf_vocab
    # option norm: 'sum', 'cosine', '**4', 'max'

    term_frequencies = {}
    for term in query:
        term_frequencies[term] = term_frequencies.get(term, 0) + 1

    for term, frequency in term_frequencies.items():
        if term in order_term:
            tf = frequency
            idf = new_idf_vocab[term]['idf']
            tf_idf_query[term] = tf * idf

    if option_norm == 'sum':
        tf_idf_sum = sum(tf_idf_query.values())
        if tf_idf_sum != 0:
            tf_idf_query = {term: tf_idf / tf_idf_sum for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'cosine':
        tf_idf_norm = math.sqrt(sum([tf_idf ** 2 for tf_idf in tf_idf_query.values()]))
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == '**4':
        tf_idf_norm = sum([tf_idf ** 4 for tf_idf in tf_idf_query.values()])
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'max':
        tf_idf_max = max(tf_idf_query.values())
        if tf_idf_max != 0:
            tf_idf_query = {term: tf_idf / tf_idf_max for term, tf_idf in tf_idf_query.items()}
    return tf_idf_query


In [ ]:
def calculate_tf_idf_queryLemmatizer(query, new_tf_sorted_indexs, new_idf_vocab, order_term, option_norm):
    tf_idf_query = {}
    query = query.lower()
    query = re.findall(r'\b\w+\b', query)

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    query = [lemmatizer.lemmatize(word) for word in query if word not in stop_words]

    # tf number appearance
    # idf get in new_idf_vocab
    # option norm: 'sum', 'cosine', '**4', 'max'

    term_frequencies = {}
    for term in query:
        term_frequencies[term] = term_frequencies.get(term, 0) + 1

    for term, frequency in term_frequencies.items():
        if term in order_term:
            tf = frequency
            idf = new_idf_vocab[term]['idf']
            tf_idf_query[term] = tf * idf

    if option_norm == 'sum':
        tf_idf_sum = sum(tf_idf_query.values())
        if tf_idf_sum != 0:
            tf_idf_query = {term: tf_idf / tf_idf_sum for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'cosine':
        tf_idf_norm = math.sqrt(sum([tf_idf ** 2 for tf_idf in tf_idf_query.values()]))
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == '**4':
        tf_idf_norm = sum([tf_idf ** 4 for tf_idf in tf_idf_query.values()])
        if tf_idf_norm != 0:
            tf_idf_query = {term: tf_idf / tf_idf_norm for term, tf_idf in tf_idf_query.items()}
    elif option_norm == 'max':
        tf_idf_max = max(tf_idf_query.values())
        if tf_idf_max != 0:
            tf_idf_query = {term: tf_idf / tf_idf_max for term, tf_idf in tf_idf_query.items()}
    return tf_idf_query

# query = "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft ."
# tf_idf_query = calculate_tf_idf_queryLemmatizer(query, new_tf_sorted_indexs, new_idf_vocab, order_term, 'sum')

## Information Retrieval Function (Finding Relevant Documents)

In [ ]:
def information_retrieval(tf_idf_query, new_idf_vocab, new_w_sorted_indexs):
    result = {}
    for term, query_weight in tf_idf_query.items():
        if term not in new_idf_vocab:
            continue
        # get doc_id by new_idf_vocab['position']
        # get w by new_w_sortd_indexs[doc_id]
        positions = list(new_idf_vocab[term]['position'])
        for position in positions:
            entry = new_w_sorted_indexs[position]
            doc_id = entry['doc_id']
            doc_weight = entry['w']
            retrieval_score = query_weight * doc_weight
            if doc_id in result:
                result[doc_id] += retrieval_score
            else:
                result[doc_id] = retrieval_score
    return result

# retrieval_scores = information_retrieval(tf_idf_query,new_idf_vocab, new_tf_sorted_indexs)
# sorted_results = sorted(retrieval_scores.items(), key=lambda x: x[1], reverse=True)
# sorted_results[:29]

## Calculate the AP/mAP of all the queries

In [ ]:
def precision(y_true, y_pred):
  prec_list = []
  rec_list = []
  count = 0
  for i in range(len(y_pred)):
    if y_pred[i] in y_true:
      count += 1
      prec_list.append(count/(i+1))
      rec_list.append(count/len(y_true))
  return prec_list, rec_list
# list prec nội suy nè
def prec_in(prec_list, rec_list):
  prec_list_in = []
  for i in range(11):
    rec = i/10
    # prec_rec = max(rec->all)
    # lấy max từ prec_rec_in
    prec_rec_in = next((x[0] for x in enumerate(rec_list) if x[1] >= rec),-1)
    # print('rec_in:', prec_rec_in)
    prec_rec = 0
    if prec_rec_in != -1:
      prec_rec = max(prec_list[prec_rec_in:])
    prec_list_in.append(prec_rec)
  return prec_list_in

In [ ]:
def MAP(list_y_true, list_y_pred, noisuy=True):
    common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
    result_dict = {}

    if noisuy:
        prec_in_data = {}
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            prec_in_list = prec_in(prec_list, rec_list)
            # print('prec_list', prec_list)
            # print('rec_list', rec_list)
            # print('prec_list_in', prec_in_list)
            result_dict[key] = sum(prec_in_list) / len(prec_in_list)
            prec_in_data[key] = prec_in_list
        return prec_in_data, result_dict, sum(result_dict.values()) / len(result_dict.values())
    else:
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            if len(prec_list) != 0:
                result_dict[key] = sum(prec_list) / len(prec_list)
            else:
                result_dict[key] = 0

    return result_dict, sum(result_dict.values()) / len(result_dict.values())

## P and R ver chang

In [ ]:
def count_P_R(list_y_true, list_y_pred):
  common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
  result_P_dict = {}
  result_R_dict = {}
  for key in common_keys:
      count = 0
      for i in range(len(list_y_pred[key])):
        if list_y_pred[key][i] in list_y_true[key]:
          count += 1
      if len(list_y_pred[key])!=0:
        result_P_dict[key] = count / len(list_y_pred[key])
      else:
        result_P_dict[key] = 0
      result_R_dict[key] = count / len(list_y_true[key])
  return result_P_dict, result_R_dict, sum(result_P_dict.values()) / len(result_P_dict.values()), sum(result_R_dict.values()) / len(result_R_dict.values())

In [ ]:
def trec_11(prec_in_data):
  trec_11 = {}
  for i in range(11):
    trec_11[i/10] = 0
    for j in prec_in_data:
      trec_11[i/10]+=prec_in_data[j][i]/len(prec_in_data)
  return trec_11

# Porter Stemmer Ver Cranfield

In [ ]:
## Loading the 'Gold' result
import os

folder_path = '/content/RES'
trueResult = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            lines = file.readlines()
            second_column = [int(line.split()[1]) for line in lines]
            key = int(file_name[:-4])  # Remove the '.txt' extension and convert to int
            trueResult[key] = second_column

trueResult = dict(sorted(trueResult.items()))

In [ ]:
# def information_retrieval(tf_idf_query, new_idf_vocab, new_w_sorted_indexs):
#     result = {}
#     for term, query_weight in tf_idf_query.items():
#         if term not in new_idf_vocab:
#             continue
#         for entry in new_w_sorted_indexs:
#             if entry['word'] == term:
#                 doc_id = entry['doc_id']
#                 doc_weight = entry['w']
#                 retrieval_score = query_weight * doc_weight
#                 if doc_id in result:
#                     result[doc_id] += retrieval_score
#                 else:
#                     result[doc_id] = retrieval_score
#     return result
# retrieval_scores = information_retrieval(tf_idf_query,new_idf_vocab, new_tf_sorted_indexs)
# sorted_results = sorted(retrieval_scores.items(), key=lambda x: x[1], reverse=True)
# sorted_results[:10]

In [ ]:
import time

# Measure total execution time
start_time = time.time()

docs, terms = preprocessPorterStemmer(sorted_data_dict)
indexs, order_term, inverse_order_term = indexing(docs, terms)

print(len(indexs))

max_id = find_max_id(docs)

## Sorting the index list in alphabetical order
sorted_indexs = sorted(indexs, key=lambda x: x['word'])
## Adding the frequencies to the terms in the index table
new_sorted_indexs = adding_frequency(docs, terms, sorted_indexs)
# Build the vocab table using the docs, terms and the sorted indexes
vocab = build_vocab(docs, terms, sorted_indexs)

# Calculate total execution time
total_timePSStopIndex = time.time() - start_time

# Print total execution time
print("Total execution time:", total_timePSStopIndex)

128879
Total execution time: 3.1580779552459717


In [ ]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')

number of word in terms: 4556
number of word in indexs: 128879
number of document: 1400
The max of doc_id: 1400


### Cranfield Porter Stem + Remove Stop

In [ ]:
new_tf_sorted_indexs = calculate_tf(docs, terms, vocab, sorted_indexs, 'binary')
new_idf_vocab = calculate_idf(docs, terms, vocab, sorted_indexs, 'inverse')
new_w_sorted_indexs = calculate_w(docs, terms, new_idf_vocab, new_tf_sorted_indexs, 'cosine')

retrievalResult = dict()

start_time = time.time()

for key, value in queries.items():
    valueTFIDF = calculate_tf_idf_queryPorterStemmer(value, new_tf_sorted_indexs, new_idf_vocab, order_term, 'cosine')

    retrieval_scores = information_retrieval(valueTFIDF,new_idf_vocab, new_tf_sorted_indexs)
    sorted_results = sorted(retrieval_scores.items(), key=lambda x: x[1], reverse=True)

    retrievalResult[key] = sorted_results

# Calculate total execution time
total_timePSStopQuery = time.time() - start_time

predResult = {}

for key, value in retrievalResult.items():
    values_first_column = [tup[0] for tup in value]
    predResult[key] = values_first_column

predResult = dict(sorted(predResult.items()))

APTrecList_PorterStem_Stop, listOfAP_PorterStem_Stop_Inter, MAP_PorterStem_Stop_Inter = MAP(trueResult, predResult)
listOfAP_PorterStem_Stop_NonInter, MAP_PorterStem_Stop_NonInter = MAP(trueResult, predResult, False)
l_p1, l_r1, p1, r1 = count_P_R(trueResult, predResult)
# Print total execution time
print("Total execution time:", total_timePSStopQuery)
print(MAP_PorterStem_Stop_NonInter)
print(MAP_PorterStem_Stop_Inter)
print(p1, r1)

Total execution time: 0.5523908138275146
0.353978722166619
0.363837230755112
0.009635067304641724 0.9614997186296037


In [ ]:
trec_11(APTrecList_PorterStem_Stop)

{0.0: 0.7198219064932468,
 0.1: 0.6962468957340252,
 0.2: 0.5814469616753831,
 0.3: 0.4736735170605677,
 0.4: 0.387431352506125,
 0.5: 0.33478682149858313,
 0.6: 0.26102106224843685,
 0.7: 0.19066954684607582,
 0.8: 0.15000320133445355,
 0.9: 0.10987808246764962,
 1.0: 0.09723019044168525}

## Cranfield Porter Stem Only

In [ ]:
import time

start_time = time.time()

docs, terms = preprocessPorterStemmerOnly(sorted_data_dict)
indexs, order_term, inverse_order_term = indexing(docs, terms)

print(len(indexs))

max_id = find_max_id(docs)

## Sorting the index list in alphabetical order
sorted_indexs = sorted(indexs, key=lambda x: x['word'])
## Adding the frequencies to the terms in the index table
new_sorted_indexs = adding_frequency(docs, terms, sorted_indexs)
# Build the vocab table using the docs, terms and the sorted indexes
vocab = build_vocab(docs, terms, sorted_indexs)

# Calculate total execution time
total_timePSIndex = time.time() - start_time

# Print total execution time
print("Total execution time:", total_timePSIndex)

222329
Total execution time: 4.941881418228149


In [ ]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')

number of word in terms: 4665
number of word in indexs: 222329
number of document: 1400
The max of doc_id: 1400


In [ ]:
new_tf_sorted_indexs = calculate_tf(docs, terms, vocab, sorted_indexs, 'binary')
new_idf_vocab = calculate_idf(docs, terms, vocab, sorted_indexs, 'inverse')
new_w_sorted_indexs = calculate_w(docs, terms, new_idf_vocab, new_tf_sorted_indexs, 'cosine')

retrievalResult = dict()

start_time = time.time()

for key, value in queries.items():
    valueTFIDF = calculate_tf_idf_queryPorterStemmerOnly(value, new_tf_sorted_indexs, new_idf_vocab, order_term, 'cosine')

    retrieval_scores = information_retrieval(valueTFIDF,new_idf_vocab, new_tf_sorted_indexs)
    sorted_results = sorted(retrieval_scores.items(), key=lambda x: x[1], reverse=True)

    retrievalResult[key] = sorted_results

total_timePSQuery = time.time() - start_time

predResult = {}

for key, value in retrievalResult.items():
    values_first_column = [tup[0] for tup in value]
    predResult[key] = values_first_column

predResult = dict(sorted(predResult.items()))

APTrecList_PorterStem, listOfAP_PorterStem_Inter, MAP_PorterStem_Inter = MAP(trueResult, predResult)
listOfAP_PorterStem_NonInter, MAP_PorterStem_NonInter = MAP(trueResult, predResult, False)
l_p2, l_r2, p2, r2 = count_P_R(trueResult, predResult)

# Print total execution time
print("Total execution time:", total_timePSQuery)
print(MAP_PorterStem_NonInter)
print(MAP_PorterStem_Inter)
print(p2, r2)

Total execution time: 5.1728174686431885
0.09707784234840108
0.10544948253182199
0.0059751919321114405 0.998962962962963


# Lemmatizer Ver Cranfield

In [ ]:
import time

start_time = time.time()

docs, terms = preprocessLematizer(sorted_data_dict)
indexs, order_term, inverse_order_term = indexing(docs, terms)

print(len(indexs))

max_id = find_max_id(docs)
## Sorting the index list in alphabetical order
sorted_indexs = sorted(indexs, key=lambda x: x['word'])
## Adding the frequencies to the terms in the index table
new_sorted_indexs = adding_frequency(docs, terms, sorted_indexs)
# Build the vocab table using the docs, terms and the sorted indexes
vocab = build_vocab(docs, terms, sorted_indexs)

# Calculate total execution time
total_timeLemmaStopIndex = time.time() - start_time

# Print total execution time
print("Total execution time:", total_timeLemmaStopIndex)

128879
Total execution time: 3.2119948863983154


In [ ]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')

number of word in terms: 6424
number of word in indexs: 128879
number of document: 1400
The max of doc_id: 1400


In [ ]:
queries[15]

'material properties of photoelastic materials .'

In [ ]:
queries[68]

'what possible techniques are available for computing the injection distribution corresponding to an isothermal transpiration cooled hemisphere .'

In [ ]:
query = queries[68]
query = query.lower()
query = re.findall(r'\b\w+\b', query)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
query = [lemmatizer.lemmatize(word) for word in query if word not in stop_words]
query

['possible',
 'technique',
 'available',
 'computing',
 'injection',
 'distribution',
 'corresponding',
 'isothermal',
 'transpiration',
 'cooled',
 'hemisphere']

In [ ]:
for i in query:
  if i in terms:
    print(i)

possible
technique
available
computing
injection
distribution
corresponding
isothermal
transpiration
cooled
hemisphere


### Cranfield Lemma + Remove Stop

---



In [ ]:
new_tf_sorted_indexs = calculate_tf(docs, terms, vocab, sorted_indexs, 'binary')
new_idf_vocab = calculate_idf(docs, terms, vocab, sorted_indexs, 'inverse')
new_w_sorted_indexs = calculate_w(docs, terms, new_idf_vocab, new_tf_sorted_indexs, 'cosine')

retrievalResult = dict()

start_time = time.time()

for key, value in queries.items():
    valueTFIDF = calculate_tf_idf_queryLemmatizer(value, new_tf_sorted_indexs, new_idf_vocab, order_term, 'cosine')

    retrieval_scores = information_retrieval(valueTFIDF,new_idf_vocab, new_tf_sorted_indexs)
    sorted_results = sorted(retrieval_scores.items(), key=lambda x: x[1], reverse=True)

    retrievalResult[key] = sorted_results

# Calculate total execution time
total_timeLemmaStopQuery = time.time() - start_time

predResult = {}

for key, value in retrievalResult.items():
    values_first_column = [tup[0] for tup in value]
    predResult[key] = values_first_column

# predResult = dict(sorted(predResult.items()))
# for i in retrievalResult:
#   print(retrievalResult[i])

APTrecList_Lemma_Stop, listOfAP_Lemma_Stop_Inter, MAP_Lemma_Stop_Inter = MAP(trueResult, predResult)
listOfAP_Lemma_Stop_NonInter, MAP_Lemma_Stop_NonInter = MAP(trueResult, predResult, False)
l_p3, l_r3, p3, r3 = count_P_R(trueResult, predResult)

# Print total execution time
print("Total execution time:", total_timeLemmaStopQuery)
print(MAP_Lemma_Stop_NonInter)
print(MAP_Lemma_Stop_Inter)
print(p3, r3)

Total execution time: 0.8435192108154297
0.35357811324286986
0.3600326648839338
0.010631994701123564 0.9572936868730649


## Lemma Only

In [ ]:
import time

start_time = time.time()

docs, terms = preprocessLemmatizerOnly(sorted_data_dict)
indexs, order_term, inverse_order_term = indexing(docs, terms)

print(len(indexs))

max_id = find_max_id(docs)
## Sorting the index list in alphabetical order
sorted_indexs = sorted(indexs, key=lambda x: x['word'])

## Adding the frequencies to the terms in the index table
new_sorted_indexs = adding_frequency(docs, terms, sorted_indexs)

# Build the vocab table using the docs, terms and the sorted indexes
vocab = build_vocab(docs, terms, sorted_indexs)

# Calculate total execution time
total_timeLemmaIndex = time.time() - start_time

# Print total execution time
print("Total execution time:", total_timeLemmaIndex)

222329
Total execution time: 2.5257585048675537


In [ ]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')

number of word in terms: 6537
number of word in indexs: 222329
number of document: 1400
The max of doc_id: 1400


In [ ]:
new_tf_sorted_indexs = calculate_tf(docs, terms, vocab, sorted_indexs, 'binary')
new_idf_vocab = calculate_idf(docs, terms, vocab, sorted_indexs, 'inverse')
new_w_sorted_indexs = calculate_w(docs, terms, new_idf_vocab, new_tf_sorted_indexs, 'cosine')

retrievalResult = dict()

start_time = time.time()

for key, value in queries.items():
    valueTFIDF = calculate_tf_idf_queryLemmatizerOnly(value, new_tf_sorted_indexs, new_idf_vocab, order_term, 'cosine')

    retrieval_scores = information_retrieval(valueTFIDF,new_idf_vocab, new_tf_sorted_indexs)
    sorted_results = sorted(retrieval_scores.items(), key=lambda x: x[1], reverse=True)

    retrievalResult[key] = sorted_results

total_timeLemmaQuery = time.time() - start_time

predResult = {}

for key, value in retrievalResult.items():
    values_first_column = [tup[0] for tup in value]
    predResult[key] = values_first_column

predResult = dict(sorted(predResult.items()))

APTrecList_Lemma, listOfAP_Lemma_Inter, MAP_Lemma_Inter = MAP(trueResult, predResult)
l_p4, l_r4, p4, r4 = count_P_R(trueResult, predResult)
listOfAP_Lemma_NonInter, MAP_Lemma_NonInter = MAP(trueResult, predResult, False)

# Print total execution time
print("Total execution time:", total_timeLemmaQuery)
print(MAP_Lemma_NonInter)
print(MAP_Lemma_Inter)
print(p4, r4)

Total execution time: 5.089728116989136
0.09416105045574764
0.1030757029383086
0.005990695802272081 0.9991953522149601


# Cranfield Blank

In [ ]:
def preprocess(docs):
    # docs is a dict {id: doc}
    terms = set()
    modified_docs = {}

    for i, doc in enumerate(docs):
        modified_doc = docs[doc].lower()
        modified_doc = re.sub(r'[^a-z0-9\s]', '', modified_doc)
        # use nltk to tokenize and filter the punctuation
        words = word_tokenize(modified_doc)
        sentence = ""
        for word in words:
            if word not in string.punctuation:
                terms.add(word)
                sentence += " " + word
        modified_docs[doc] = sentence

    return modified_docs, terms

In [ ]:
start_time = time.time()

docs, terms = preprocess(sorted_data_dict)
indexs, order_term, inverse_order_term = indexing(docs, terms)

print(len(indexs))

max_id = find_max_id(docs)
## Sorting the index list in alphabetical order
sorted_indexs = sorted(indexs, key=lambda x: x['word'])

## Adding the frequencies to the terms in the index table
new_sorted_indexs = adding_frequency(docs, terms, sorted_indexs)

# Build the vocab table using the docs, terms and the sorted indexes
vocab = build_vocab(docs, terms, sorted_indexs)

# Calculate total execution time
total_timeBaseIndex = time.time() - start_time

# Print total execution time
print("Total execution time:", total_timeBaseIndex)

225121
Total execution time: 1.6690175533294678


In [ ]:
print(f'number of word in terms: {len(terms)}')
print(f'number of word in indexs: {len(indexs)}')
print(f'number of document: {len(docs)}')
print(f'The max of doc_id: {max_id}')

number of word in terms: 7908
number of word in indexs: 225121
number of document: 1400
The max of doc_id: 1400


In [ ]:
new_tf_sorted_indexs = calculate_tf(docs, terms, vocab, sorted_indexs, 'binary')
new_idf_vocab = calculate_idf(docs, terms, vocab, sorted_indexs, 'inverse')
new_w_sorted_indexs = calculate_w(docs, terms, new_idf_vocab, new_tf_sorted_indexs, 'cosine')

retrievalResult = dict()

start_time = time.time()

for key, value in queries.items():
    valueTFIDF = calculate_tf_idf_query(value, new_tf_sorted_indexs, new_idf_vocab, order_term, 'cosine')

    retrieval_scores = information_retrieval(valueTFIDF,new_idf_vocab, new_tf_sorted_indexs)
    sorted_results = sorted(retrieval_scores.items(), key=lambda x: x[1], reverse=True)

    retrievalResult[key] = sorted_results

# Calculate total execution time
total_timeBaseQuery = time.time() - start_time

predResult = {}

for key, value in retrievalResult.items():
    values_first_column = [tup[0] for tup in value]
    predResult[key] = values_first_column

predResult = dict(sorted(predResult.items()))

APTrecList_Stop, listOfAP_Stop_Inter, MAP_Stop_Inter = MAP(trueResult, predResult)
l_p5, l_r5, p5, r5 = count_P_R(trueResult, predResult)
listOfAP_Stop_NonInter, MAP_Stop_NonInter = MAP(trueResult, predResult, False)

# Print total execution time
print("Total execution time:", total_timeBaseQuery)
print(MAP_Stop_NonInter)
print(MAP_Stop_Inter)
print(p5, r5)

Total execution time: 5.173335790634155
0.0874221823766076
0.09585905467350601
0.0060077890179921056 0.9960595497458242




---

# Thống kê

---



In [ ]:
import pandas as pd

# Define your data
q1 = ["PorterStem + Stop", MAP_PorterStem_Stop_Inter, MAP_PorterStem_Stop_NonInter, p1, r1, total_timePSStopIndex, total_timePSStopQuery]
q2 = ["PorterStem", MAP_PorterStem_Inter, MAP_PorterStem_NonInter, p2, r2, total_timePSIndex, total_timePSQuery]
q3 = ["Lemma + Stop", MAP_Lemma_Stop_Inter, MAP_Lemma_Stop_NonInter, p3, r3, total_timeLemmaStopIndex, total_timeLemmaStopQuery]
q4 = ["Lemma", MAP_Lemma_Inter, MAP_Lemma_NonInter, p4, r4, total_timeLemmaIndex, total_timeLemmaQuery]
q5 = ["No Preprocess", MAP_Stop_Inter, MAP_Stop_NonInter, p5, r5, total_timeBaseIndex, total_timeBaseQuery]

col = ["Type", "Inter", "NonInter", "P", "R", "Index Time", "Query Time"]

# Create the DataFrame
df = pd.DataFrame([q1, q2, q3, q4, q5], columns=col)

# Round the numeric columns to 4 decimal places
numeric_cols = ["Inter", "NonInter", "P", "R", "Index Time", "Query Time"]
df[numeric_cols] = df[numeric_cols].round(4)

# Print the rounded DataFrame
df


,Type,Inter,NonInter,P,R,Index Time,Query Time
0,PorterStem + Stop,0.3638,0.3540,0.0096,0.9615,3.1581,0.5524
1,PorterStem,0.1054,0.0971,0.0060,0.9990,4.9419,5.1728
2,Lemma + Stop,0.3600,0.3536,0.0106,0.9573,3.2120,0.8435
3,Lemma,0.1031,0.0942,0.0060,0.9992,2.5258,5.0897
4,No Preprocess,0.0959,0.0874,0.0060,0.9961,1.6690,5.1733
